<h1>Overpass API Notebook</h1>

<p>This notebook is dedicated to call the overpass API and extract data for our ADS Project. The data will enrich our rental price prediction models with relevant geo-spacial data_ in the vicitnity of a given apartment such as restaurants, supermarkets and other points of interest.<p> 

In [10]:
import json
import requests
import pandas as pd


# Request all restaurants, cafes, and supermarkets in the Kanton of Zurich, Switzerland
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area["alt_name"="Kanton Zürich"]->.a;
(node["amenity"="restaurant"](area.a);
 node["amenity"="cafe"](area.a);
 node["shop"="supermarket"](area.a););
out center;
"""


# Server response
response = requests.get(overpass_url, params={'data': overpass_query})

data = response.json()

# Extract relevant data from the JSON response
places = []
for element in data['elements']:
    if 'tags' in element:
        place_type = element['tags'].get('amenity') or element['tags'].get('shop')
        places.append({
            'id': element['id'],
            'type': place_type,
            'lat': element['lat'],
            'lon': element['lon'],
            'name': element['tags'].get('name', ''),
            'brand': element['tags'].get('brand', ''),
            'address_city': element['tags'].get('addr:city', ''),
            'address_zip': element['tags'].get('addr:postcode', ''),
            'address_street': element['tags'].get('addr:street', ''),
            'address_number': element['tags'].get('addr:housenumber', ''),
            'website': element['tags'].get('website', '')
            

        })

# Convert the list of dictionaries to a pandas DataFrame
places_df = pd.DataFrame(places)

# Display the DataFrame
places_df


,id,type,lat,lon,name,brand,address_city,address_zip,address_street,address_number,website
0,34039212,restaurant,47.495518,8.741556,Hallenbadrestaurant Geiselweid,,,,,,
1,34040796,restaurant,47.483065,8.705704,Schwimmbadrestaurant Töss,,,,,,
2,39947904,supermarket,47.376732,8.542161,Coop,Coop,Zürich,8001,Bahnhofbrücke,1,
3,48932835,supermarket,47.375020,8.522895,Migros,Migros,Zürich,8004,Wengistrasse,7,https://filialen.migros.ch/de/migros-supermark...
4,57370928,restaurant,47.245360,8.726042,Metzg,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...
3566,11755283869,cafe,47.358640,8.523193,Cake Friends,,,,,,
3567,11764916519,restaurant,47.319772,8.521702,Restaurant Soodhof,,,,,,
3568,11775729223,cafe,47.374081,8.531948,Bang On,,Zürich,8004,Kasernenstrasse,15,https://www.bang-on.ch/
3569,11780149080,restaurant,47.418242,8.555513,,,,,,,


In [11]:
types = ['restaurant', 'cafe', 'supermarket']

for i in types:
    print(i ,':', (places_df['type'] == i).sum())

restaurant : 2337
cafe : 608
supermarket : 624


In [1]:
import pandas as pd

filepath = 'TablesDB/Location_v2.csv'

location_df = pd.read_csv(filepath, delimiter=",")
location_df

,LocationId,Street,ZIP,longitude,latitude
0,78753131-6d78-4d1e-a0e5-fc8b5f430570,NaN,6598.0,NaN,NaN
1,f0c2bc0e-55ab-4eb1-98fa-edcb4a0ee01e,via albonago 43,6962.0,8.974113,46.013360
2,fbdcde66-0fd7-4304-a83e-70ecbb0f6ae7,Via San Gottardo 41,6500.0,9.268200,45.583280
3,2d7451a3-d7e1-4dc7-babf-ab237f2c02e4,Via F. Zorzi 17,6500.0,9.015443,46.187900
4,2bde1434-f480-4d46-9b05-99e0c0659671,Via San Gottardo,6900.0,NaN,NaN
...,...,...,...,...,...
11347,2dba2402-d23f-4ab8-9bb6-702bbe28cb4e,Chemin de la Petite-Californie,1222.0,6.207831,46.242178
11348,daed4779-995c-4c34-85e7-4b269d8bf5cc,Chemin William-Barbey,1292.0,6.149006,46.241745
11349,faddd38c-e9fc-4f4d-bf09-a41818cc620f,Rue du Temple 5,1236.0,2.352571,48.857414
11350,84d332ee-e1a8-4b5f-a432-82819fffe330,Chemin de la Pierre-à-Bochet,1226.0,6.214858,46.199271


In [3]:
pip install haversine

Note: you may need to restart the kernel to use updated packages.


In [8]:
import haversine as hs   
from haversine import Unit
 
def calculate_distance (lat1, lon1, lat2, lon2):
    loc1=(lat1, lon1)
    loc2=(lat2, lon2)
 
    distance=hs.haversine(loc1,loc2,unit=Unit.METERS)
    return distance

location1 = [47.373032740890785, 8.498880746229132]
location2 = [47.370284238918465, 8.509635802377217]

dist = calculate_distance(location1[0], location1[1], location2[0], location2[1])
dist




865.6616515214281

In [20]:
import json
import requests
import pandas as pd


# Request all restaurants, cafes,  in the Kanton of Zurich, Switzerland
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area["alt_name"="Kanton Zürich"]->.a;
(node["shop"="supermarket"](area.a););
out center;
"""


# Server response
response = requests.get(overpass_url, params={'data': overpass_query})

data = response.json()

# Extract relevant data from the JSON response
places = []
for element in data['elements']:
    if 'tags' in element:
        place_type = element['tags'].get('amenity') or element['tags'].get('shop')
        places.append({
            'id': element['id'],
            'type': place_type,
            'lat': element['lat'],
            'lon': element['lon'],
            'name': element['tags'].get('name', ''),
            'brand': element['tags'].get('brand', ''),
            'address_city': element['tags'].get('addr:city', ''),
            'address_zip': element['tags'].get('addr:postcode', ''),
            'address_street': element['tags'].get('addr:street', ''),
            'address_number': element['tags'].get('addr:housenumber', ''),
            'website': element['tags'].get('website', '')
            

        })

# Convert the list of dictionaries to a pandas DataFrame
supermarket_df = pd.DataFrame(places)

# Display the DataFrame
supermarket_df

,id,type,lat,lon,name,brand,address_city,address_zip,address_street,address_number,website
0,39947904,supermarket,47.376732,8.542161,Coop,Coop,Zürich,8001,Bahnhofbrücke,1,
1,48932835,supermarket,47.375020,8.522895,Migros,Migros,Zürich,8004,Wengistrasse,7,https://filialen.migros.ch/de/migros-supermark...
2,70656485,supermarket,47.491253,8.733981,Mini-Discount,,,,,,
3,70656488,supermarket,47.491874,8.706448,Migros,Migros,Winterthur,8406,Zürcherstrasse,102,https://filialen.migros.ch/de/migros-supermark...
4,75749133,supermarket,47.340967,8.530601,ALDI,ALDI,Zürich,8038,Albisstrasse,81,https://www.aldi-suisse.ch/
...,...,...,...,...,...,...,...,...,...,...,...
622,11247689896,supermarket,47.229496,8.667794,Alnatura Super Natur Markt,Alnatura,Wädenswil,8820,Oberdorfstrasse,32,
623,11281203294,supermarket,47.395911,8.608894,Breitibach Market,,,,,,https://breitibach-market.ch
624,11416701493,supermarket,47.388883,8.483650,Denner,Denner,Zürich,8048,Bachmattstrasse,59,https://www.denner.ch/
625,11416840012,supermarket,47.345356,8.534797,ALDI Suisse,ALDI,Zürich,8038,Mythenquai,353,https://www.aldi-suisse.ch/


In [23]:
import haversine as hs   
from haversine import Unit
 
def count_nearby_poi (apartment_lat, apartment_lon, poi, radius):
    
    count = 0
    for index, point in poi.iterrows():
        # Calculate distance between apartment and point of interest
        distance = hs.haversine((apartment_lat, apartment_lon), (point['lat'], point['lon']), unit=Unit.METERS)
        if distance <= radius:
            count += 1
    return count
 
apartment = [47.373032740890785, 8.498880746229132]
counter = count_nearby_poi(apartment[0], apartment[1], supermarket_df, 300)

counter

1